In [ ]:
# This file was created by the Image Processing Research Group of University Federico II of Naples ('GRIP-UNINA')
# and adapted by IDLab-MEDIA, Ghent University - imec, in collaboration with GRIP-UNINA

In [1]:
from time import time
import os
import tensorflow as tf
from tensorflow import keras as ks

import splicebuster
import splicebuster.main_blind_function
import splicebuster.noiseprint.noiseprint_blind_concat

D:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
#main_folder = f"/project_ghent/public/comprint"
# main_folder = f"/project_ghent/comprint"
main_folder = f"../"

In [7]:
# Load model
def get_model(modelname, main_folder):
    if modelname == "full-jpg-ps-full":
        model = ks.models.load_model('%s/models/Comprint_Siamese_Full_jpg_ps_full' % main_folder)
        print("Model %s loaded" % modelname)
    elif modelname == "net":
        # For noiseprint, load different model depending on jpg QF
        print("Don't load model %s because it's dependent on QF" % modelname)
        noiseprint_model_folder = f"%s/models/noiseprint_nets" % (main_folder)
        model = "%s/%s" % (noiseprint_model_folder, modelname)
        print(model)
    else:
        print("Model %s not found" % modelname)
        model = None
    return model

In [8]:
modelnames = ["full-jpg-ps-full"] # Comprint
modelnames = ["net"] # Noiseprint
modelnames = ["full-jpg-ps-full", "net"] # Fusion of Comprint + Noiseprint

In [9]:
# Load model(s)
models = []
modelname_full = ""
if len(modelnames) > 1:
    modelname_full = "_concat"
for modelname in modelnames:
    model = get_model(modelname, main_folder)
    models.append(model)
    modelname_full = "%s_%s" % (modelname_full, modelname)
    
print("Full modelname: %s" % modelname_full)



Model full-jpg-ps-full loaded
Don't load model net because it's dependent on QF
..//models/noiseprint_nets/net
Full modelname: _concat_full-jpg-ps-full_net


# Set input

In [10]:
### SET INPUT FOLDERS
# main_folder_data = "/project_ghent/public/comprint"
main_folder_data = "../"
input_folder = "%s/data/examples_input" % main_folder_data
output_folder = "%s/data/examples_output" % main_folder_data

In [23]:
### SET INPUT
# You can set another input file here
input_filename = "lady-gaga-fake.PNG"
#input_filename = 

In [24]:
# Convert filenames to full paths for the output files
def get_output_filenames(input_filename, input_folder, output_folder):
    input_file = f"%s/%s" % (input_folder, input_filename)
    output_filename = "%s/%s%s" % (input_filename, input_filename, modelname_full)

    fingerprint_files = []
    output_filename_base = "%s/%s" % (input_filename, input_filename)
    output_file_base = f"%s/%s" % (output_folder, output_filename_base)
    for modelname in modelnames:
        fingerprint_file = f"%s_%s_res.png" % (output_file_base, modelname)
        fingerprint_files.append(fingerprint_file)

        # Build concatenated filename
        output_filename = "%s_%s" % (output_filename, modelname)

    output_file = f"%s/%s" % (output_folder, output_filename)
    #output_file_npz = f"%s.npz" % (output_file)
    heatmap_file = f"%s_heatmap.png" % (output_file)
    return input_file, output_file, fingerprint_files, heatmap_file

In [25]:
input_file, output_file, fingerprint_files, heatmap_file = get_output_filenames(input_filename, input_folder, output_folder)

print(input_file)
print(output_file)
print(fingerprint_files)
print(heatmap_file)

..//data/examples_input/lady-gaga-fake.PNG
..//data/examples_output/lady-gaga-fake.PNG/lady-gaga-fake.PNG_concat_full-jpg-ps-full_net_full-jpg-ps-full_net
['..//data/examples_output/lady-gaga-fake.PNG/lady-gaga-fake.PNG_full-jpg-ps-full_res.png', '..//data/examples_output/lady-gaga-fake.PNG/lady-gaga-fake.PNG_net_res.png']
..//data/examples_output/lady-gaga-fake.PNG/lady-gaga-fake.PNG_concat_full-jpg-ps-full_net_full-jpg-ps-full_net_heatmap.png


In [26]:
# Run fingerprint extraction and forgery localization algorithm
print("Start detection on %s" % input_file)
t = time()
splicebuster.main_blind_function.extract_heatmap_concat(input_file, output_file, models, fingerprint_files, heatmap_file)
elapsed_time = time() - t
print(f"Finished in %f seconds" % elapsed_time)

Start detection on ..//data/examples_input/lady-gaga-fake.PNG
INFO:tensorflow:Restoring parameters from ..//models/noiseprint_nets/net_jpg101/model


INFO:tensorflow:Restoring parameters from ..//models/noiseprint_nets/net_jpg101/model


 252x372 small 101
Heatmap saved to ..//data/examples_output/lady-gaga-fake.PNG/lady-gaga-fake.PNG_concat_full-jpg-ps-full_net_full-jpg-ps-full_net_heatmap.png
Finished in 2.253999 seconds


In [36]:
# Run fingerprint extraction and forgery localization algorithm
print("Start detection on %s" % input_file)
t = time()
splicebuster.main_blind_function.extract_heatmap_concat(input_file, output_file, models, fingerprint_files, heatmap_file)
elapsed_time = time() - t
print(f"Finished in %f seconds" % elapsed_time)

Start detection on /project_ghent/public/comprint/data/examples_input/facehub-fake.png
INFO:tensorflow:Restoring parameters from /project_ghent/comprint/models/noiseprint_nets/net_jpg101/model
 312x696 small 101
Heatmap saved to /project_ghent/public/comprint/data/examples_output/facehub-fake.png/facehub-fake.png_concat_full-jpg-ps-full_net_full-jpg-ps-full_net_heatmap.png
Finished in 10.166313 seconds
